In [1]:
import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from transformers import BertTokenizer,TFBertModel,TFBertForSequenceClassification

In [2]:
emotionpush_data_path = r'/home/ning/dataset'
bert_path = r"/home/ning/bert_conf"
cache_dir=r'/home/ning/bert_conf/bert-base-uncased-cache'

ckpt_path = r'/home/ning/models.ckpt'

train_df = pd.read_pickle(os.path.join(emotionpush_data_path,'emotionpush_train.pkl'))
test_df = pd.read_pickle(os.path.join(emotionpush_data_path,'emotionpush_test.pkl'))

train_df = train_df[train_df['emotion'].isin([ 'neutral', 'joy', 'sadness', 'anger'])]
test_df = test_df[test_df['emotion'].isin([ 'neutral', 'joy', 'sadness', 'anger'])]

tokenizer = BertTokenizer.from_pretrained(os.path.join(bert_path,'vocab_idea.txt'))
# model = TFBertForSequenceClassification.from_pretrained(os.path.join(ckpt_path,'friendsbert_no_pretrained.h5'),config = os.path.join(ckpt_path,'config.json'),num_labels=4)
#原版
model = TFBertForSequenceClassification.from_pretrained(os.path.join(bert_path,'tf_model.h5'),config = os.path.join(bert_path,'config.json'),num_labels=4)


Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
Some weights of the model checkpoint at /home/ning/bert_conf/tf_model.h5 were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at /home/ning/bert_conf/tf_model.h5 and are newly initialized: ['classifier', 'dropout_37']
You should probably TRAIN this model on a down-stream task 

In [3]:
def encode_sentence_with_speaker(speaker,utterance,tokenizer,sos):
    if sos:
        tokens = ['[CLS]']
    else:
        tokens = []

    if utterance in [';)',':)','<3',':(','LMAO','LOL','OMG','','lol','lmao','omg']:
        return tokenizer.encode([utterance.upper()])
        
    
    if speaker not in ['other','None']:
        spk_token = '['+speaker+']'
        tokens.append(spk_token)
        tokens.append('[says]')
        tokens.extend(list(tokenizer.tokenize(utterance)))
        tokens.append('[SEP]')
    
    else:
        tokens.extend(list(tokenizer.tokenize(utterance)))
    
    return tokenizer.convert_tokens_to_ids(tokens)
    
    


def bert_encode(dataframe, tokenizer,single=False):
    
    if not single:
        num_examples = len(dataframe.index)
        sentence1 = tf.ragged.constant([encode_sentence_with_speaker(s[0],s[1],tokenizer,True) for s in dataframe.values])

        sentence2 = tf.ragged.constant([encode_sentence_with_speaker(s[2],s[3],tokenizer,False) for s in dataframe.values])

        input_word_ids = tf.concat([sentence1, sentence2], axis=-1)

        input_mask = tf.ones_like(input_word_ids).to_tensor()

        type_s1 = tf.zeros_like(sentence1)
        type_s2 = tf.ones_like(sentence2)
        input_type_ids = tf.concat([type_s1, type_s2], axis=-1).to_tensor()


        inputs = {
            'input_ids': input_word_ids.to_tensor(),
            'attention_mask': input_mask,
            'token_type_ids': input_type_ids}
    
    else:
        num_examples = len(dataframe.index)
        input_word_ids = tf.ragged.constant([encode_sentence_with_speaker(s[0],s[1],tokenizer,True) for s in dataframe.values])
        
        input_mask = tf.ones_like(input_word_ids).to_tensor()
        
        input_type_ids = tf.zeros_like(input_word_ids).to_tensor()
        
        inputs = {
            'input_ids': input_word_ids.to_tensor(),
            'attention_mask': input_mask,
            'token_type_ids': input_type_ids}
        
        
    return inputs

In [4]:
train_target = train_df['emotion']
test_target = test_df['emotion']

#数据增强
# addition_anger = train_df[train_df['emotion']=='anger']
# addition_sadness = train_df[train_df['emotion']=='sadness']
# addition_joy = train_df[train_df['emotion']=='joy']
# for i in range(3):
#     addition_anger = pd.concat([addition_anger,train_df[train_df['emotion']=='anger']], axis=0)
# addition_sadness = pd.concat([addition_sadness,addition_anger], axis=0)

# addition_sadness = pd.concat([addition_sadness,addition_sadness], axis=0)
# addition_sadness = pd.concat([addition_sadness,addition_sadness], axis=0)
# addition_joy = pd.concat([addition_joy,addition_sadness], axis=0)

# result = pd.concat([addition_joy,addition_joy,addition_joy,addition_joy], axis=0)
# train_df = result.sample(frac=0.5)


train_features = bert_encode(train_df, tokenizer,single = False)
test_features = bert_encode(test_df, tokenizer,single = False)

ems = train_target.unique()
def convert(emotion):
    return np.where(ems == emotion)[0][0]

train_labels = np.array(list(map(convert,train_target)))
test_labels = np.array(list(map(convert,test_target)))

In [15]:
def NLL(y_true,y_pred):
    
    sample_weight = np.array([1.2761053029348783, 24.830882352941178, 6.051971326164875, 92.1])
    node1 = tf.nn.softmax(y_pred)
    node2 = sample_weight*node1
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    return loss(y_true,node2)/sample_weight.sum()


optimizer = tf.keras.optimizers.Adam(learning_rate=2.5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['accuracy'])
model.fit(
      train_features,train_labels,
    
#       validation_data=(test_features,test_labels),
      batch_size=16,
      epochs=1)

634/634 [==============================] - 220s 346ms/step - loss: 0.0026 - accuracy: 0.8895


In [16]:
m,p = test(model,test_features,test_labels)

[[1863    6   32    1]
 [  55   43    8    0]
 [ 187    2  237    0]
 [  29    0    0    1]]
            neutral   sadness       joy     anger  Overall
Precision  0.873008  0.843137  0.855596  0.500000      NaN
Recall     0.979495  0.405660  0.556338  0.033333      NaN
F1-score   0.923191  0.547771  0.674253  0.062500  0.87013


In [13]:
model.save_pretrained('/home/ning/models.ckpt/ChatBert_nopretrain_finetune')

In [5]:


epochs = 1
batch_size = 16
num_labels = 4
def calc_confusion_matrix(matrix,predict,true):

    for i in range(len(predict)):
        matrix[true[i]][predict[i]] += 1
    return matrix

def calc_acc(matrix):
    r = 0
    for i,label in enumerate(matrix):
        r += matrix[i][i]
    
    return r/sum(sum(matrix))
    
def calc_performance(matrix):
    rights = matrix[[range(4)],[range(4)]]
    
    Ps = rights/matrix.sum(axis=0)
    Rs = rights/matrix.sum(axis=1)
    
    f1_scores = 2*Ps*Rs/(Ps+Rs)
    
    micro_f1_scores = rights.sum()/sum(matrix.sum(axis=0))

    overall_p = np.array([[np.nan],[np.nan],[micro_f1_scores]])
    
    perfromence_ = np.concatenate((Ps,Rs,f1_scores),axis=0)
    perfromence = np.concatenate((perfromence_,overall_p),axis=1)
    
    df = pd.DataFrame(perfromence,columns=['neutral', 'sadness', 'joy', 'anger','Overall'],index=['Precision','Recall','F1-score'])
    
    
    return df
    
def NLL(y_true,y_pred):
    
    sample_weight = np.array([1.2761053029348783, 24.830882352941178, 6.051971326164875, 92.1])
    node1 = tf.nn.softmax(y_pred)
    node2 = sample_weight*node1
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    return loss(y_true,node2)/sample_weight.sum()

loss_object= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# loss_object = NLL

# train_data_size = len(train_labels)
# steps_per_epoch = int(train_data_size / batch_size)
# num_train_steps = steps_per_epoch * epochs
# warmup_steps = int(epochs * train_data_size * 0.1 / batch_size)

# optimizer = nlp.optimization.create_optimizer(
#     2.5e-6, num_train_steps=num_train_steps, num_warmup_steps=warmup_steps)



def train(model,train_f,train_l,test_f,test_l,one_step = False):

    performance = pd.DataFrame()
    batch_size = 8
    num_labels = 4
    epochs = 5
    weight = np.array([[sum(train_target==e) for e in ems]]*batch_size)
    weight = weight[0].sum()/weight
    sample_weight = tf.constant(weight,dtype=tf.float32)
    
    confusion_matrix = np.zeros((num_labels,num_labels),dtype=np.float32)
    optimizer = tf.keras.optimizers.Adam(learning_rate=2.5e-6)
    
    for i in range(epochs):
        print("Training for epoch {}".format(i))
        for step in range(len(train_l)//batch_size):
            train_f_batch = {'input_ids': train_f['input_ids'][batch_size*step:batch_size*(step+1)],
                    'attention_mask': train_f['attention_mask'][batch_size*step:batch_size*(step+1)],
                    'token_type_ids': train_f['token_type_ids'][batch_size*step:batch_size*(step+1)]}
            train_l_batch = train_l[batch_size*step:batch_size*(step+1)]

            with tf.GradientTape() as tape:
                prediction = model(train_f_batch,training = True)
                pred_label = np.argmax(prediction[0].numpy(),axis=1)
                
                loss_value  = loss_object(y_true=train_l_batch, y_pred=prediction)
                
                
            grads = tape.gradient(loss_value, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            
            if one_step:
                return None,None
            
            
            if not step%50:
                print("Epoch {:03d}: Step:{:03d}//{:03d} Loss: {:.3f} ".format(i,step,len(train_l)//batch_size,loss_value))
                confusion_matrix,performance = test(model,test_features,test_labels,batch_size = 16,num_labels = 4)
                
    return confusion_matrix,performance



def test(model,test_f,test_l,batch_size = 16,num_labels = 4):
    
    confusion_matrix = np.zeros((num_labels,num_labels),dtype=np.float32)
    
    for step_ in range(len(test_l)//batch_size):
        test_f_batch = {'input_ids': test_f['input_ids'][batch_size*step_:batch_size*(step_+1)],
                'attention_mask': test_f['attention_mask'][batch_size*step_:batch_size*(step_+1)],
                'token_type_ids': test_f['token_type_ids'][batch_size*step_:batch_size*(step_+1)]}
        test_l_batch = test_l[batch_size*step_:batch_size*(step_+1)]

        prediction = model(test_f_batch,training = False)
        pred_label = np.argmax(prediction[0].numpy(),axis=1)

        confusion_matrix = calc_confusion_matrix(confusion_matrix,pred_label,test_l_batch)

    print(confusion_matrix.astype(np.int32))
    performance = calc_performance(confusion_matrix)

    print(performance)    
    return confusion_matrix.astype(np.int32),performance


In [ ]:
confusion_matrix,performance = train(model,train_features,train_labels,test_features,test_labels)

Training for epoch 0
Epoch 000: Step:000//1266 Loss: 1.319 
[[ 346 1400    0  156]
 [  11   89    0    6]
 [  47  343    0   36]
 [   3   25    0    2]]
            neutral   sadness  joy     anger   Overall
Precision  0.850123  0.047927  NaN  0.010000       NaN
Recall     0.181914  0.839623  0.0  0.066667       NaN
F1-score   0.299697  0.090678  NaN  0.017391  0.177354


<ipython-input-5-7b3d2177ac82>:20: RuntimeWarning: invalid value encountered in true_divide
  Ps = rights/matrix.sum(axis=0)


Epoch 000: Step:050//1266 Loss: 0.825 
[[1886   15    0    1]
 [ 104    2    0    0]
 [ 401   25    0    0]
 [  30    0    0    0]]
            neutral   sadness  joy  anger   Overall
Precision  0.779017  0.047619  NaN    0.0       NaN
Recall     0.991588  0.018868  0.0    0.0       NaN
F1-score   0.872542  0.027027  NaN    NaN  0.766234


<ipython-input-5-7b3d2177ac82>:20: RuntimeWarning: invalid value encountered in true_divide
  Ps = rights/matrix.sum(axis=0)
<ipython-input-5-7b3d2177ac82>:23: RuntimeWarning: invalid value encountered in true_divide
  f1_scores = 2*Ps*Rs/(Ps+Rs)


Epoch 000: Step:100//1266 Loss: 1.024 
[[1898    4    0    0]
 [ 106    0    0    0]
 [ 413   13    0    0]
 [  30    0    0    0]]
            neutral  sadness  joy  anger   Overall
Precision  0.775644      0.0  NaN    NaN       NaN
Recall     0.997897      0.0  0.0    0.0       NaN
F1-score   0.872844      NaN  NaN    NaN  0.770292


...

Epoch 004: Step:750//1266 Loss: 0.951 
[[1749   12  141    0]
 [  40   44   22    0]
 [ 107    2  317    0]
 [  14    9    7    0]]
            neutral   sadness       joy  anger   Overall
Precision  0.915707  0.656716  0.650924    NaN       NaN
Recall     0.919558  0.415094  0.744131    0.0       NaN
F1-score   0.917629  0.508671  0.694414    NaN  0.856331


<ipython-input-5-7b3d2177ac82>:20: RuntimeWarning: invalid value encountered in true_divide
  Ps = rights/matrix.sum(axis=0)


Epoch 004: Step:800//1266 Loss: 0.014 
[[1843    7   52    0]
 [  58   35   13    0]
 [ 184    1  241    0]
 [  21    7    2    0]]
            neutral   sadness       joy  anger   Overall
Precision  0.875119  0.700000  0.782468    NaN       NaN
Recall     0.968980  0.330189  0.565728    0.0       NaN
F1-score   0.919661  0.448718  0.656676    NaN  0.859984


<ipython-input-5-7b3d2177ac82>:20: RuntimeWarning: invalid value encountered in true_divide
  Ps = rights/matrix.sum(axis=0)


Epoch 004: Step:850//1266 Loss: 0.147 


In [10]:
test(model,test_features,test_labels)

[[1519  173   51  159]
 [  81   10   11    4]
 [ 339   32   32   23]
 [  24    2    2    2]]
            neutral   sadness       joy     anger   Overall
Precision  0.773816  0.046083  0.333333  0.010638       NaN
Recall     0.798633  0.094340  0.075117  0.066667       NaN
F1-score   0.786028  0.061920  0.122605  0.018349  0.634334


(array([[1519,  173,   51,  159],
        [  81,   10,   11,    4],
        [ 339,   32,   32,   23],
        [  24,    2,    2,    2]], dtype=int32),
             neutral   sadness       joy     anger   Overall
 Precision  0.773816  0.046083  0.333333  0.010638       NaN
 Recall     0.798633  0.094340  0.075117  0.066667       NaN
 F1-score   0.786028  0.061920  0.122605  0.018349  0.634334)

In [13]:
weights = [sum(train_df['emotion']==e) for e in ems]

In [14]:
weights

[7939, 408, 1674, 110]

In [15]:

weights = [sum(train_df['emotion']==e) for e in ems]
weights = [sum(weights)/i for i in weights]
weights

[1.2761053029348783, 24.830882352941178, 6.051971326164875, 92.1]

In [17]:
ems

array(['neutral', 'sadness', 'joy', 'anger'], dtype=object)

In [8]:
addition = train_df[train_df['emotion']!='neutral']
result = pd.concat([addition,addition,addition,train_df, addition], axis=0)
result.sample(frac=1)

In [9]:
train_df[train_df['emotion']=='joy'].shape


(1674, 5)

In [10]:

train_df[train_df['emotion']=='sadness'].shape


(408, 5)

In [23]:
addition_anger = train_df[train_df['emotion']=='anger']
addition_sadness = train_df[train_df['emotion']=='sadness']
addition_joy = train_df[train_df['emotion']=='joy']
for i in range(3):
    addition_anger = pd.concat([addition_anger,train_df[train_df['emotion']=='anger']], axis=0)
addition_sadness = pd.concat([addition_sadness,addition_anger], axis=0)

addition_sadness = pd.concat([addition_sadness,addition_sadness], axis=0)
addition_sadness = pd.concat([addition_sadness,addition_sadness], axis=0)
addition_joy = pd.concat([addition_joy,addition_sadness], axis=0)

train_df = pd.concat([train_df,addition_joy,addition_joy,addition_joy], axis=0)


In [29]:
train_df[train_df['emotion']=='neutral'].shape

(7939, 5)

In [1]:
train_df[train_df['utterance']=='lol']

NameError: name 'train_df' is not defined

In [19]:
train_df[train_df['utterance']==':)'][['utterance','emotion']]#,'causal_utterance','emotion']]

,utterance,emotion
3590,:),joy
5479,:),joy
5480,:),joy
5653,:),joy
7030,:),joy
7790,:),joy
8435,:),joy
9901,:),joy
9909,:),joy


In [17]:
train_df.head()

,speaker,utterance,previous_speaker,causal_utterance,emotion
0,other,No,None,None,neutral
1,other,Why him?,other,No,neutral
2,other,Yeah one tix is fine,other,Why him?,neutral
3,other,person_0,other,Yeah one tix is fine,neutral
5,other,If you know anyone else who wants a tix let me...,other,Wait like buy from him?,neutral
